In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

# Dataset paths
train_dir = r"Datasets\train"
test_dir = r"Datasets\test"

# Step 1: Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=False
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=False
)

Found 557 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [4]:
# Step 2: Build feature extractor (CNN without classification head)
feature_extractor = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten()
])

C:\Users\Lakshya\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Step 3: Extract features
X_train = feature_extractor.predict(train_data)
y_train = train_data.classes

X_test = feature_extractor.predict(test_data)
y_test = test_data.classes

print("✅ Features extracted:", X_train.shape, X_test.shape)

C:\Users\Lakshya\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


28/28 ━━━━━━━━━━━━━━━━━━━━ 13s 460ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 451ms/step
✅ Features extracted: (557, 36992) (140, 36992)


In [6]:
# Step 4: Train SVM classifier
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [7]:
# Step 5: Evaluate
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n🎯 SVM Accuracy on test data: {accuracy:.4f}")


🎯 SVM Accuracy on test data: 0.6500


In [15]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image

# Path to a single test image
img_path = r"Datasets\test\dogs\dog_147.jpg"   

# 1. Load and preprocess the image (same size as training)
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # shape: (1, 150, 150, 3)

# 2. Extract features using CNN feature extractor
features = feature_extractor.predict(img_array)

# 3. Predict with SVM
prediction = svm_model.predict(features)[0]

# 4. Map result
label_map = {0: "Cat 🐱", 1: "Dog 🐶"}
print("Prediction:", label_map[prediction])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Prediction: Dog 🐶
